In [24]:
import requests
import pandas as pd
import xmltodict
import time
from tqdm import tqdm

In [25]:
ROOT_PATH = "https://boardgamegeek.com/xmlapi2/"

In [26]:
# read data_info.xlsx 'games' sheet into dataframe
games_df = pd.read_excel("data_info.xlsx", sheet_name="games", header=None, names=["bgg_url"])
games_df.head()

,bgg_url
0,https://boardgamegeek.com/boardgame/13/catan
1,https://boardgamegeek.com/boardgame/162886/spi...
2,https://boardgamegeek.com/boardgame/248562/mag...
3,https://boardgamegeek.com/boardgame/285774/mar...
4,https://boardgamegeek.com/boardgame/167791/ter...


In [27]:
games_df["game_id"] = games_df["bgg_url"].str.extract(r"boardgame/(\d+)/")
games_df.head()

,bgg_url,game_id
0,https://boardgamegeek.com/boardgame/13/catan,13
1,https://boardgamegeek.com/boardgame/162886/spi...,162886
2,https://boardgamegeek.com/boardgame/248562/mag...,248562
3,https://boardgamegeek.com/boardgame/285774/mar...,285774
4,https://boardgamegeek.com/boardgame/167791/ter...,167791


In [28]:
# Save ids to list, filter out NaNs
game_ids = games_df["game_id"].dropna().tolist()
game_ids[:5]

['13', '162886', '248562', '285774', '167791']

In [29]:
# Get bgg_url with NaN as game_id and extract game_id from url it is after boardgameexpansion/
expansion_ids = games_df.loc[games_df["game_id"].isna(), "bgg_url"].str.extract(r"boardgameexpansion/(\d+)/")[0].tolist()
expansion_ids

['287988', '321259', '91072', '315200']

In [30]:
ids_as_str = ",".join(game_ids)
results = requests.get(f"{ROOT_PATH}thing?id={ids_as_str}")

In [31]:
res_json = xmltodict.parse(results.content)

In [32]:
len(res_json["items"]["item"])

234

In [33]:
parsed_game_data = []
for game in tqdm(res_json["items"]["item"]):
    name_state = game["name"]
    if isinstance(name_state, list):
        name = name_state[0]["@value"]
    else:
        name = name_state["@value"]
    parsed_game_data.append(dict(
        name=name,
        release_year=int(game["yearpublished"]["@value"]),
        category=[
            category["@value"] for category in 
            game["link"] if category["@type"] == "boardgamecategory"],
        mechanic=[
            mechanic["@value"] for mechanic in
            game["link"] if mechanic["@type"] == "boardgamemechanic"],
        players_min=int(game["minplayers"]["@value"]),
        players_max=int(game["maxplayers"]["@value"]),
        playtime_min=int(game["minplaytime"]["@value"]),
        playtime_max=int(game["maxplaytime"]["@value"]),
        min_age=int(game["minage"]["@value"]),
        is_expansion=False
    ))

100%|██████████| 234/234 [00:00<00:00, 47020.89it/s]


In [36]:
parsed_game_data[2]

{'name': 'Mage Knight: Ultimate Edition',
 'release_year': 2018,
 'category': ['Adventure', 'Exploration', 'Fantasy', 'Fighting'],
 'mechanic': ['Card Play Conflict Resolution',
  'Cooperative Game',
  'Deck, Bag, and Pool Building',
  'Dice Rolling',
  'Grid Movement',
  'Hand Management',
  'Modular Board',
  'Open Drafting',
  'Role Playing',
  'Solo / Solitaire Game',
  'Tile Placement',
  'Variable Player Powers'],
 'players_min': 1,
 'players_max': 5,
 'playtime_min': 150,
 'playtime_max': 150,
 'min_age': 14,
 'is_expansion': False}

In [37]:
import json
# Save data to json file
with open("./data/bgg_data.json", "w") as f:
    json.dump(parsed_game_data, f)